## SFTTrainer Example - Finetuning with Sagemaker

This notebook demonstrates basic user flow for SFT Finetuning from a model available in Sagemaker Jumpstart.
Information on available models on jumpstart: https://docs.aws.amazon.com/sagemaker/latest/dg/jumpstart-foundation-models-latest.html

### Setup and Configuration

Initialize the environment by importing necessary libraries and configuring AWS credentials

In [1]:
from sagemaker.train.sft_trainer import SFTTrainer
from sagemaker.train.common import TrainingType
from sagemaker.core.training.configs import InputData
from rich import print as rprint
from rich.pretty import pprint
from sagemaker.core.resources import ModelPackage

import boto3
from sagemaker.core.helper.session_helper import Session


# Beta endpoint configuration
api_endpoint = "https://api.sagemaker.gamma.us-west-2.ml-platform.aws.a2z.com"

# Create SageMaker clients with beta endpoints
sm_client = boto3.client('sagemaker', endpoint_url=api_endpoint, region_name="us-west-2")

# Create SageMaker session with beta endpoints
sagemaker_session = Session(sagemaker_client=sm_client)




[11/24/25 16:00:25] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=868434;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=681134;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/botocore/credentials.py#1364\1364]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/rsareddy/Library/Application Support/sagemaker/config.yaml


[11/24/25 16:00:26] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=226866;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=449259;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/botocore/credentials.py#1364\1364]8;;\

## Finetuning with Jumpstart base model

### Create SFTTrainer
**Required Parameters** 

* `model`: base_model id on Sagemaker Hubcontent that is available to finetune (or) ModelPackage artifacts

**Optional Parameters**
* `training_type`: Choose from TrainingType Enum(sagemaker.modules.train.common) either LORA OR FULL.
* `model_package_group_name`: ModelPackage group name or ModelPackageGroup
* `mlflow_resource_arn`: MLFlow app ARN to track the training job
* `mlflow_experiment_name`: MLFlow app experiment name(str)
* `mlflow_run_name`: MLFlow app run name(str)
* `training_dataset`: Training Dataset - either Dataset ARN or S3 Path of the dataset (Please note these are required for a training job to run, can be either provided via Trainer or .train())
* `validation_dataset`: Validation Dataset - either Dataset ARN or S3 Path of the dataset
* `s3_output_path`: S3 path for the trained model artifacts

In [2]:
# For fine-tuning 
sft_trainer = SFTTrainer(
    model="meta-textgeneration-llama-3-2-1b-instruct", # Union[str, ModelPackage]
    training_type=TrainingType.LORA, 
    model_package_group_name="test-finetuned-models-gamma", # Make it Optional
    mlflow_resource_arn="arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/mmlu-eval-experiment",  # Optional[str] - MLflow app ARN (auto-resolved if not provided), can accept name and search in the account
    mlflow_experiment_name="test-finetuned-models-exp", # Optional[str]
    mlflow_run_name="test-finetuned-models-run", # Optional[str]
    training_dataset="arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/sft-test-dataset/0.0.6", #Optional[]
    s3_output_path="s3://open-models-testing-pdx/output",
    sagemaker_session=sagemaker_session,
    role="arn:aws:iam::052150106756:role/Admin"
)


us-west-2
gamma


### Discover and update Finetuning options

Each of the technique and model has overridable hyperparameters that can be finetuned by the user.

In [3]:
print("Default Finetuning Options:")
pprint(sft_trainer.hyperparameters.to_dict()) # rename as hyperparameters

Default Finetuning Options:


{
│   'data_path': '',
│   'global_batch_size': '8',
│   'learning_rate': '0.0001',
│   'lora_alpha': '32',
│   'lr_warmup_ratio': '0.1',
│   'max_epochs': '5',
│   'mlflow_run_id': '',
│   'mlflow_tracking_uri': '',
│   'model_name_or_path': 'meta-llama/Llama-3.2-1B-Instruct',
│   'name': 'example-name-zha9i',
│   'output_path': '',
│   'results_directory': '',
│   'resume_from_path': '',
│   'train_val_split_ratio': '0.9',
│   'training_data_name': '',
│   'validation_data_name': '',
│   'validation_data_path': ''
}

In [4]:
# To update any hyperparameter, simply assign the value, example:
sft_trainer.hyperparameters.global_batch_size=16

#### Start SFT training


In [4]:
training_job = sft_trainer.train(
    wait=True,
)

[11/24/25 16:00:38] INFO     Training Job Name:                                                  ]8;id=69435;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/sft_trainer.py\sft_trainer.py]8;;\:]8;id=253488;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/sft_trainer.py#106\106]8;;\
                             meta-textgeneration-llama-3-2-1b-instruct-sft-20251124160038                          

                    INFO     MLflow resource ARN:                                             ]8;id=577558;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/common_utils/finetune_utils.py\finetune_utils.py]8;;\:]8;id=192238;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/common_utils/finetune_utils.py#435\435]8;;\
                             arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/                      
                             mmlu-eval-experiment                                                                  

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/rsareddy/Library/Application Support/sagemaker/config.yaml


                    INFO     Creating training_job resource.                                     ]8;id=103208;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/resources.py\resources.py]8;;\:]8;id=670509;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/resources.py#35539\35539]8;;\

Output()

[11/24/25 16:15:38] INFO     Final Resource Status: Completed                                    ]8;id=865518;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/resources.py\resources.py]8;;\:]8;id=601194;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/resources.py#35872\35872]8;;\

In [ ]:
#In order to skip waiting and monitor the training Job later

'''
training_job = sft_trainer.train(
    wait=False,
)
'''

In [5]:
pprint(training_job)

TrainingJob(
│   training_job_name='meta-textgeneration-llama-3-2-1b-instruct-sft-20251123162832',
│   training_job_arn='arn:aws:sagemaker:us-west-2:052150106756:training-job/meta-textgeneration-llama-3-2-1b-instruct-sft-20251123162832',
│   processing_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   tuning_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   labeling_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   auto_ml_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   model_artifacts=ModelArtifacts(
│   │   s3_model_artifacts='s3://open-models-testing-pdx/output/meta-textgeneration-llama-3-2-1b-instruct-sft-20251123162832/output/model'
│   ),
│   training_job_output=TrainingJobOutput(
│   │   s3_training_job_output='s3://open-models-testing-pdx/output/meta-textgeneration-llama-3-2-1b-instruct-sft-20251123162832/output/output'
│   ),
│   training_job_status='Completed',
│   secondary_status='Completed',
│   failure_reason=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   hyper_parameters={
│   │   'global_batch_size': '8',
│   │   'learning_rate': '0.0001',
│   │   'lora_alpha': '32',
│   │   'lr_warmup_ratio': '0.1',
│   │   'max_epochs': '5',
│   │   'model_name_or_path': 'meta-llama/Llama-3.2-1B-Instruct',
│   │   'name': 'example-name-4xpu2',
│   │   'train_val_split_ratio': '0.9'
│   },
│   algorithm_specification=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   role_arn='arn:aws:iam::052150106756:role/Admin',
│   input_data_config=[
│   │   Channel(
│   │   │   channel_name='train',
│   │   │   data_source=DataSource(
│   │   │   │   s3_data_source=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   │   │   file_system_data_source=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   │   │   dataset_source=DatasetSource(
│   │   │   │   │   dataset_arn='arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/sft-test-dataset/0.0.6'
│   │   │   │   )
│   │   │   ),
│   │   │   content_type=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   │   compression_type='None',
│   │   │   record_wrapper_type='None',
│   │   │   input_mode=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   │   shuffle_config=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   │   enable_ffm=False
│   │   )
│   ],
│   output_data_config=OutputDataConfig(
│   │   s3_output_path='s3://open-models-testing-pdx/output',
│   │   kms_key_id='',
│   │   compression_type='NONE',
│   │   remove_job_name_from_s3_output_path=False,
│   │   disable_model_upload=False,
│   │   channels=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>
│   ),
│   resource_config=ResourceConfig(
│   │   instance_type=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   instance_count=0,
│   │   volume_size_in_gb=0,
│   │   volume_kms_key_id=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   keep_alive_period_in_seconds=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   capacity_reservation_ids=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   instance_groups=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   capacity_schedules_config=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   training_plan_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   instance_placement_config=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>
│   ),
│   warm_pool_status=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   vpc_config=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   stopping_condition=StoppingCondition(
│   │   max_runtime_in_seconds=86400,
│   │   max_wait_time_in_seconds=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   max_p

### View any Training job details

We can get any training job details and its status with TrainingJob.get(...)

In [8]:
from sagemaker.core.resources import TrainingJob

response = TrainingJob.get(training_job_name="meta-textgeneration-llama-3-2-1b-instruct-sft-20251123162832")
pprint(response)

TrainingJob(
│   training_job_name='meta-textgeneration-llama-3-2-1b-instruct-sft-20251123162832',
│   training_job_arn='arn:aws:sagemaker:us-west-2:052150106756:training-job/meta-textgeneration-llama-3-2-1b-instruct-sft-20251123162832',
│   processing_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   tuning_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   labeling_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   auto_ml_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   model_artifacts=ModelArtifacts(
│   │   s3_model_artifacts='s3://open-models-testing-pdx/output/meta-textgeneration-llama-3-2-1b-instruct-sft-20251123162832/output/model'
│   ),
│   training_job_output=TrainingJobOutput(
│   │   s3_training_job_output='s3://open-models-testing-pdx/output/meta-textgeneration-llama-3-2-1b-instruct-sft-20251123162832/output/output'
│   ),
│   training_job_status='Completed',
│   secondary_status='Completed',
│   failure_reason=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   hyper_parameters={
│   │   'global_batch_size': '8',
│   │   'learning_rate': '0.0001',
│   │   'lora_alpha': '32',
│   │   'lr_warmup_ratio': '0.1',
│   │   'max_epochs': '5',
│   │   'model_name_or_path': 'meta-llama/Llama-3.2-1B-Instruct',
│   │   'name': 'example-name-4xpu2',
│   │   'train_val_split_ratio': '0.9'
│   },
│   algorithm_specification=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   role_arn='arn:aws:iam::052150106756:role/Admin',
│   input_data_config=[
│   │   Channel(
│   │   │   channel_name='train',
│   │   │   data_source=DataSource(
│   │   │   │   s3_data_source=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   │   │   file_system_data_source=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   │   │   dataset_source=DatasetSource(
│   │   │   │   │   dataset_arn='arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/sft-test-dataset/0.0.6'
│   │   │   │   )
│   │   │   ),
│   │   │   content_type=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   │   compression_type='None',
│   │   │   record_wrapper_type='None',
│   │   │   input_mode=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   │   shuffle_config=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   │   enable_ffm=False
│   │   )
│   ],
│   output_data_config=OutputDataConfig(
│   │   s3_output_path='s3://open-models-testing-pdx/output',
│   │   kms_key_id='',
│   │   compression_type='NONE',
│   │   remove_job_name_from_s3_output_path=False,
│   │   disable_model_upload=False,
│   │   channels=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>
│   ),
│   resource_config=ResourceConfig(
│   │   instance_type=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   instance_count=0,
│   │   volume_size_in_gb=0,
│   │   volume_kms_key_id=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   keep_alive_period_in_seconds=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   capacity_reservation_ids=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   instance_groups=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   capacity_schedules_config=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   training_plan_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   instance_placement_config=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>
│   ),
│   warm_pool_status=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   vpc_config=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   stopping_condition=StoppingCondition(
│   │   max_runtime_in_seconds=86400,
│   │   max_wait_time_in_seconds=<sagemaker.core.utils.utils.Unassigned object at 0x11938dee0>,
│   │   max_p

## Continued Finetuning (or) Finetuning on Model Artifacts

#### Discover a ModelPackage and get its details

In [5]:
from rich import print as rprint
from rich.pretty import pprint
from sagemaker.core.resources import ModelPackage, ModelPackageGroup

#model_package_iter = ModelPackage.get_all(model_package_group_name="test-finetuned-models-gamma")
model_package = ModelPackage.get(model_package_name="arn:aws:sagemaker:us-west-2:052150106756:model-package/test-finetuned-models-gamma/68")

pprint(model_package)

ModelPackage(
│   model_package_name=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   model_package_group_name='test-finetuned-models-gamma',
│   model_package_version=68,
│   model_package_registration_type='Logged',
│   model_package_arn='arn:aws:sagemaker:us-west-2:052150106756:model-package/test-finetuned-models-gamma/68',
│   model_package_description=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   creation_time=datetime.datetime(2025, 11, 23, 16, 34, 0, 391000, tzinfo=tzlocal()),
│   inference_specification=InferenceSpecification(
│   │   containers=[
│   │   │   ContainersItem(
│   │   │   │   model_data_url=None,
│   │   │   │   image=None,
│   │   │   │   nearest_model_name=None,
│   │   │   │   model_data_source=ModelDataSource(),
│   │   │   │   is_checkpoint=False,
│   │   │   │   base_model=BaseModelInfo(hub_content_name='meta-textgeneration-llama-3-2-1b-instruct')
│   │   │   )
│   │   ]
│   ),
│   source_algorithm_specification=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   validation_specification=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   model_package_status='Completed',
│   model_package_status_details=ModelPackageStatusDetails(validation_statuses=[], image_scan_statuses=[]),
│   certify_for_marketplace=False,
│   model_approval_status=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   created_by=UserContext(
│   │   user_profile_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   │   user_profile_name=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   │   domain_id=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   │   iam_identity=IamIdentity(
│   │   │   arn='arn:aws:sts::052150106756:assumed-role/Admin/SageMaker',
│   │   │   principal_id='AROAQYJDDPKCM5Q7HXZYK:SageMaker',
│   │   │   source_identity=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>
│   │   )
│   ),
│   metadata_properties=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   model_metrics=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   deployment_specification=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   last_modified_time=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   last_modified_by=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   approval_description=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   domain=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   task=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   sample_payload_url=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   sample_payload_content_type=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   customer_metadata_properties=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   drift_check_baselines=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   additional_inference_specifications=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   skip_model_validation=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   source_uri=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   security_config=ModelPackageSecurityConfig(
│   │   kms_key_id=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>
│   ),
│   model_card=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   model_life_cycle=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>
)

#### Create Trainer

Trainer creation is same as above Finetuning Section except for `model`'s input is ModelPackage(previously trained artifacts)

In [6]:
# For fine-tuning 
sft_trainer = SFTTrainer(
    model=model_package, # Union[str, ModelPackage]
    training_type=TrainingType.LORA, 
    model_package_group_name="test-finetuned-models-gamma", # Make it Optional
    mlflow_resource_arn="arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/mmlu-eval-experiment",  # Optional[str] - MLflow app ARN (auto-resolved if not provided), can accept name and search in the account
    mlflow_experiment_name="test-finetuned-models-exp", # Optional[str]
    mlflow_run_name="test-finetuned-models-run", # Optional[str]
    training_dataset="arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/sft-test-dataset/0.0.6", #Optional[]
    s3_output_path="s3://open-models-testing-pdx/output",
    sagemaker_session=sagemaker_session,
    role="arn:aws:iam::052150106756:role/Admin"
)


#### Start the Training

In [4]:
training_job = sft_trainer.train(
    wait=True,
)

╭────────────────────────────────── Training Job Status ───────────────────────────────────╮
│  TrainingJob Name      meta-textgeneration-llama-3-2-1b-instruct-sft-20251123044523      │
│  MLFlow URL            ]8;id=468044;https://t-eq86xu3xab1s.us-west-2.experiments.sagemaker.aws/auth?authToken=eyJhbGciOiJIUzI1NiJ9.eyJhdXRoVG9rZW5JZCI6ImUxOTViYjRkLTA0YjUtNDJkMC05ZDQxLTQ5OTY2MzdjYTE2ZCIsImZhc0NyZWRlbnRpYWxzIjoiQWdWNDdKaDlNaHRibHJPWjFxbjZ0SkhXTjZZS25wZ3YwOUtrN1RPSWZMb2NMWWdBWHdBQkFCVmhkM010WTNKNWNIUnZMWEIxWW14cFl5MXJaWGtBUkVGMU5URjNhbXh2ZDBKRWVEUmpPVGd4VEd4SVQxTkxkVXhwUmpFMU1FeG9TWEZMSzJSTVluSnhaRVk1TUROV2RXeFRSelpaZW0wclpHdGtVWE52ZFNzMlp6MDlBQUVBQjJGM2N5MXJiWE1BUzJGeWJqcGhkM002YTIxek9uVnpMWGRsYzNRdE1qbzFPVEF4T0RNM016azFNRFE2YTJWNUx6ZzNabUUxTVdReUxURTRNRGt0TkdVMFl5MWhObVV6TFRRNFpXWTNNelk1WW1NM1lnQzRBUUlCQUhndFR2ZzEyNVZtTm50WE4vTEVZV1dzREMrSk1KRzNQSUl1eU9PYTB5SU5FZ0ZMVElWQ2RQb0NsbXVoR1lva0ZsT2JBQUFBZmpCOEJna3Foa2lHOXcwQkJ3YWdiekJ0QWdFQU1HZ0dDU3FHU0liM0RRRUhBVEFlQmdsZ2hrZ0JaUU1FQVM0d0VRUU1pMjZOeGZFUW5vZGphRk4xQWdFUWdEdUw3NUE0Y2lWM2t3NWt0VjM4dDBvNHdCTXIxS3kzT1E1Y3RDTUFwdlB0eXdGMStwdjF6VWl3alRhQk90NGVrczgxTGFMajVvS0NWZnBaYXdJQUFCQUFhSGFMVW5mUUVoSUxLZmtuMkVYTjBybENLengzeUY3OXRrVEFoeVdQOUZSTzYzL1p1dkJOVmpsL3B3UElnSUd3Ly8vLy93QUFBQUVBQUFBQUFBQUFBQUFBQUFFQUFBUkhoUHpLZnRCc09Rd2JrTEhjMkJURjF2eVNQbUF2RXJ5Vkk3azgxclJ5a3VOUm9tK243aDd2U2hzczBvaklSMy9mZEtVWWEwV0U1WE9GakViOVR2TGJwN1FDcXV1cVN5Z21BWlg1YU05L2dQY1RGSlFNYU96TGpNNGYyOFp1aTU0R3FtOUdxbVo0aE1ONFU5Y0lhZnVrbFg4a3BPQWw0RmsyQW5OOEVHNE00TXEyYWRHZXl0OHREQkUxZm51L1R1dmlVWXVON2ZSL0JUQ2YzMUFxZVFhMkVYU05vVmJrY0pCQWR1M3BKalVpdmhHUEpHTzVqWWV0S0l2S0w4VTRSR3k3NlhIc3FIdVZPZHR4ZXkzWHNwcFJTeE9SNzZTd282T3g5NVNwY09SRksvZFhPRmRRTldncDhHWHExWS94WVRLQ1IxVTNwaHpTVjV3ZVFzUDZmVGFHUHpZZkxYcHhNeVFLYnN2UDY1MDFqSm9hVTd2SDJaMXRyODBKajNmQ1hkNWpqSUMvTTJ3cmhMUldxNFlIQnRYMjVJaEsyNGNlSUZqNm5ybUdxSTRnV1lSNVlNWnlkSmVHUzhPSTllZFBsbFh3dTkvTkVlV2VubmlnV094YzRDQkRVY2FiNFdmRWsyK2hGQ2IrK3pwckIrS21SRXkvaFRGdVJTODBJeXFVUjRwTHgxQk5BS2VsOUkvbU5nd1M5ejBHcC9EV0YyeThGQS9lU0pHdWZSZlorRTJoV3pObkhROWFSYzREUEp4ang5YkJaeHZrYmxUMmlPcmY4dE5RU1JrTVU4dlQyajNHK3dBc3hpVW1aNmVCaUxoYStqYTFXRnFadGo0QnVDTEk3K0VJYnZib2hWZjBnWHlCdGE5NW5xczZXeEdKV2JvS0pINHR4UjBia2NIZ0x4NWhTWVlhSFRnUDNLYmtibGVYVU5CdW5ISkc4bm1KTmgyOUR0dWxPUjM5Q0psbkRJWklMV2xaWm9iOEdFMFhXQmpuK3BlbjVTd1c1NTVlVmxGaU9wSmROT1gvazRSeHZ0ZTI4RFlyaG5uY3grejNqeXlleE9BK3huUkVLNnZvTW1Tc0pwZnpsa0dJTDFuOHJSMm1zK3BXQW9LMXNaQTNsUUorRXRyOThoTlRKbjR0UDJRU0NEZXVmR0dUUU9sOEFnNGFWbjJjUERFVTBpRDN1S3dnaFV2aW1JazRPbFFPRTV2eUNYQURwd3hKLzdESTFweUhwa2g5bGU4djNseVR0ckkyeXJJSndHSUpLbDYwRklSWnpVWE52V1NmcC8wYlJkemtVNkwwRWRuKzhZaXFGTndBSWFWb0JQRFEySnJDREtENTZJbndWcEJSSUJGTUx3OWxmTFU1Y1oxeHNlN1IrdlJWaFRHNGIvTHNsSi9TcjVtMGc4V1JndlBoZVdVSE9wYkdHNXhHbFN6T1hzT3k3SWxMNzZiU2crV2tnU1BnQjRsRFdmbjFaSDJ4bzJ3eHozNkc3bklZa0dqMzBEclRPMW00N0t3c3lFODhxa0JKWWc1WjU0aWVjQmVITjNaUXc5OWgxWHJtSks0QXZ0QzRFS293c0VaanA0a24vdFdWaXdJMWhFR0N5SktnSW1oRjQzK0FFUGpCbzdMNVppZmlxQkxvWUtZREQwbCs0RFBWckd6dmQrYU1jajVOcENPeFl3OFEySERST3JIQnBaMXE1SXNKemtIZnJrdndDWXVNcm1wYkRYbkwwdU8xQUhBaVM1V1JOczZ6WjJJMjNldWl4TXdLM2luRHNNOUJKSUdkdmpVVDUyRjdXRDZ0a3o3b0x0VU5wSEtUR0VVVEdPM1BDRHVaNlBoOHJPWGxqOVErUVZsRjRuSWZPektwOWtGQU1EVlRzTHVWRUgwRlp5OHEvSFRVODNSOUFtVVFCMG5qMDUwckVxc045SmRwendhNDB2UVMyMTdQNkVNMU5uT2ZuaGFDbUVHMG9sK3FBODBZd1FCbk1HVUNNRXhlME1taUExeDlIVjRBSkMzNG52bnZTT250TEtXdDlkczNoeE5JanBsdWRqMmRESk41U1h1L3NCYTd3blRBZ0FJeEFQQ2d2U01ONTkzbjZoSWVoTmQ2SFBrK1FFTTQyRFo4d2NsTkUyeGlscXVWdnUyREpVU1VKNjZVRU9NRmovVFY2QT09IiwiY2lwaGVyVGV4dCI6IkFRSUJBSGd0VHZnMTI1Vm1ObnRYTi9MRVlXV3NEQytKTUpHM1BJSXV5T09hMHlJTkVnR1FiV1hIN1hvaWJvVHpNT01McVpVUUFBQUFvakNCbndZSktvWklodmNOQVFjR29JR1JNSUdPQWdFQU1JR0lCZ2txaGtpRzl3MEJCd0V3SGdZSllJWklBV1VEQkFFdU1CRUVESWxuS00rRUlsSzJvWTI1WndJQkVJQmIvY05GdlhLVzV6TzJRNGs1cG85V1RSOGNYNFpXOUFyQUJVRk1sUC9udk9JZzQrK3dWZ1Z2SVA4enFWektvRXBpdVdjRG1IaXZjbTdsZDdkczlzeHZXTThzWTRiN1ZzTEl3L21JQ21BZk1ZZElCekgxTkZnZlAyMVI2Zz09Iiwic3ViIjoiYXJuOmF3czpzYWdlbWFrZXI6dXMtd2VzdC0yOjA1MjE1MDEwNjc1NjptbGZsb3ctdHJhY2tpbmctc2VydmVyL21tbHUtZXZhbC1leHBlcmltZW50IiwiaWF0IjoxNzYzOTAx

In [7]:
training_job = sft_trainer.train(
    wait=True,
)

[11/24/25 16:22:32] INFO     Training Job Name:                                                  ]8;id=653891;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/sft_trainer.py\sft_trainer.py]8;;\:]8;id=546423;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/sft_trainer.py#106\106]8;;\
                             meta-textgeneration-llama-3-2-1b-instruct-sft-20251124162232                          

                    INFO     MLflow resource ARN:                                             ]8;id=601553;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/common_utils/finetune_utils.py\finetune_utils.py]8;;\:]8;id=508676;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/train/common_utils/finetune_utils.py#435\435]8;;\
                             arn:aws:sagemaker:us-west-2:052150106756:mlflow-tracking-server/                      
                             mmlu-eval-experiment                                                                  

                    INFO     Creating training_job resource.                                     ]8;id=528835;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/resources.py\resources.py]8;;\:]8;id=954181;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/resources.py#35539\35539]8;;\

Output()

[11/24/25 16:39:35] INFO     Final Resource Status: Completed                                    ]8;id=164796;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/resources.py\resources.py]8;;\:]8;id=189605;file:///Users/rsareddy/workplace/virtual_envs/sagemaker-v3/lib/python3.12/site-packages/sagemaker/core/resources.py#35872\35872]8;;\

In [8]:
pprint(training_job)

TrainingJob(
│   training_job_name='meta-textgeneration-llama-3-2-1b-instruct-sft-20251124162232',
│   training_job_arn='arn:aws:sagemaker:us-west-2:052150106756:training-job/meta-textgeneration-llama-3-2-1b-instruct-sft-20251124162232',
│   processing_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   tuning_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   labeling_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   auto_ml_job_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   model_artifacts=ModelArtifacts(
│   │   s3_model_artifacts='s3://open-models-testing-pdx/output/meta-textgeneration-llama-3-2-1b-instruct-sft-20251124162232/output/model'
│   ),
│   training_job_output=TrainingJobOutput(
│   │   s3_training_job_output='s3://open-models-testing-pdx/output/meta-textgeneration-llama-3-2-1b-instruct-sft-20251124162232/output/output'
│   ),
│   training_job_status='Completed',
│   secondary_status='Completed',
│   failure_reason=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   hyper_parameters={
│   │   'global_batch_size': '8',
│   │   'learning_rate': '0.0001',
│   │   'lora_alpha': '32',
│   │   'lr_warmup_ratio': '0.1',
│   │   'max_epochs': '5',
│   │   'model_name_or_path': 'meta-llama/Llama-3.2-1B-Instruct',
│   │   'name': 'example-name-zha9i',
│   │   'train_val_split_ratio': '0.9'
│   },
│   algorithm_specification=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   role_arn='arn:aws:iam::052150106756:role/Admin',
│   input_data_config=[
│   │   Channel(
│   │   │   channel_name='train',
│   │   │   data_source=DataSource(
│   │   │   │   s3_data_source=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   │   │   │   file_system_data_source=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   │   │   │   dataset_source=DatasetSource(
│   │   │   │   │   dataset_arn='arn:aws:sagemaker:us-west-2:052150106756:hub-content/AIRegistry/DataSet/sft-test-dataset/0.0.6'
│   │   │   │   )
│   │   │   ),
│   │   │   content_type=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   │   │   compression_type='None',
│   │   │   record_wrapper_type='None',
│   │   │   input_mode=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   │   │   shuffle_config=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   │   │   enable_ffm=False
│   │   )
│   ],
│   output_data_config=OutputDataConfig(
│   │   s3_output_path='s3://open-models-testing-pdx/output',
│   │   kms_key_id='',
│   │   compression_type='NONE',
│   │   remove_job_name_from_s3_output_path=False,
│   │   disable_model_upload=False,
│   │   channels=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>
│   ),
│   resource_config=ResourceConfig(
│   │   instance_type=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   │   instance_count=0,
│   │   volume_size_in_gb=0,
│   │   volume_kms_key_id=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   │   keep_alive_period_in_seconds=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   │   capacity_reservation_ids=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   │   instance_groups=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   │   capacity_schedules_config=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   │   training_plan_arn=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   │   instance_placement_config=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>
│   ),
│   warm_pool_status=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   vpc_config=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   stopping_condition=StoppingCondition(
│   │   max_runtime_in_seconds=86400,
│   │   max_wait_time_in_seconds=<sagemaker.core.utils.utils.Unassigned object at 0x11792da60>,
│   │   max_p